In [ ]:
from pathlib import Path

data_dir = Path("../data")
file = data_dir / "The-Gale-Encyclopedia-of-Medicine-3rd-Edition.pdf"

In [30]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_text_from_pdf(path):
    loader = DirectoryLoader(
        path=path,
        glob="*.pdf",
        loader_cls=PyPDFLoader 
    )

    document = loader.load()

    return document

In [31]:
extracted_data = extract_text_from_pdf(data_dir)

In [48]:
extracted_data[1]

Document(metadata={'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'creator': 'Adobe Acrobat 6.0', 'creationdate': '2006-10-16T20:19:33+02:00', 'moddate': '2006-10-16T22:03:45+02:00', 'source': '../data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition.pdf', 'total_pages': 4505, 'page': 1, 'page_label': 'ii'}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION')

In [44]:
# only needs source & page_content

from langchain.schema import Document


def filter_source_and_page_content(docs):
    filtered_docs: list[Document] = []

    for doc in docs:
        src = doc.metadata.get("source")
        filtered_docs.append(
            Document(
                metadata={"source": src},
                page_content=doc.page_content
            )
        )
    
    return filtered_docs

In [46]:
filtered_docs = filter_source_and_page_content(extracted_data)
filtered_docs[1]

Document(metadata={'source': '../data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition.pdf'}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION')